In [1]:
from Designer import DesignNotes, SST
import math
sind = lambda deg: math.sin(math.radians(deg))  # return sin of angle expressed in degrees
cosd = lambda deg: math.cos(math.radians(deg))  # return cos of angle expressed in degrees

In [2]:
import pint                  # setup to use the module for computing with units
ureg = pint.UnitRegistry()
mm = ureg['mm']              # define symbols for the units used below
inch = ureg['inch']
kN = ureg['kN']
MPa = ureg['MPa']
ureg.default_format = '~P'

In [3]:
class PartMeta(type):
    
    def __call__(cls,*args,**kwargs):
        raise TypeError("It is not possible to create an instance of this class: "+repr(cls))
    
    def __getitem__(cls,keys):
        if not type(keys) == type(()):
            keys = (keys,)
        ##print(cls,keys)
        newdct = {}
        cls_ns = None
        dct = cls.__ns__()
        for names in keys:
            for k in names.split(','):
                k = k.strip()
                if not k:
                    continue
                if '=' in k:
                    k,e = k.split('=',1)
                    k = k.strip()
                    if cls_ns is None:
                        cls_ns = (globals(),cls.ns())
                    v = eval(e,*cls_ns)
                    newdct[k] = v
                else:
                    newdct[k] = dct[k]
        newdct['__doc__'] = dct.get('__doc__')
        newname = cls.__name__ + '_Partial'
        return PartMeta(newname,cls.__bases__,newdct)

    def __enter__(cls):
        """Add all attributes/values to the set of global variables.
        Save enough state so that they can be restored when the context
        manager exits."""
        if not hasattr(cls,'__saved'):
            cls.__saved = []
        dct = cls.__dict__
        _new = []                # save a list of newly added variables
        _old = {}                # remember values of those that already exist in ns.
        ns = get_ipython().user_ns  # get the ns for the user
        for k,v in dct.items():
            if k in ns:
                _old[k] = ns[k]
            else:
                _new.append(k)
            ns[k] = v
        cls.__saved.append((_new,_old))
##        print('Push:',dct.keys(),_new,_old)
        return cls
    
    def __exit__(cls,*l):
        """When the context exits, restore the global values to what they
        were before entering."""
        _new,_old = cls.__saved.pop()
##        print('Pop:',_new,_old)
        ns = get_ipython().user_ns  # get the ns for the user
        for k,v in _old.items():
            ns[k] = v              # restore old values
        for k in _new:
            del ns[k]              # or delete them if they were newly created
        if not cls.__saved:
            del cls.__saved
        return False              # to re-raise exceptions
    
    def __ns__(cls):
        """Return the namespace."""
        ans = {}
        for c in cls.__mro__:
            if c in (Part,object):
                continue
            for k,v in c.__dict__.items():
                if not k.startswith('_'):
                    if k not in ans:
                        ans[k] = v
        return ans

    def __show__(cls,keys=None):
        """Show variables in same form as show() function. If keys is None,
        show all with _doc first.  keys can be like in show - ie, expressions,
        scales, label=expr, etc.."""
        v = cls.ns()
        if keys is None:
            pairs = sorted([(k.lower(),k) for k in v.keys()])
            keys = ','.join([o for k,o in pairs])
        show(keys,data=v)

class Part(metaclass=PartMeta):
    
    pass
     
def makePart(cls):
    """Returns an object of type Part from the class definition and class attributes
    of 'cls'.  Intended to be used as a decorator so we can use class definitions
    to build parts (syntactic sugar)."""
    dct = cls.__dict__
    bases = cls.__bases__
    newdct = {k:v for k,v in dct.items() if not k.startswith('__')}
    return PartMeta(cls.__name__,bases,newdct)


In [4]:
class Bolts(Part):
    'Bolts'   # bolt group is the same on the tongue plate and on the gusset plate.
    grade = 'ASTM A325'
    size = '3/4"'
    Fu = 825*MPa
    d = (3/4*inch).to(mm)
    hole_type = 'punched'
    hd = 22*mm            # hole diameter
    ha = hd + 2*mm        # hole allowance
    threads_intercepted = True
    nlines = 2            # a line is perpendicular to load
    nperline = 3          # number of bolts in each line
    g = 75*mm             # gauge (perpendicular to load)
    s = 75*mm             # spacing (parallel to load)

class Welds(Part):
    'Welds'
    grade = 'E49xx'
    Xu = 490*MPa
    matching = True

class Plates(Part):
    'Plates'
    grade = 'CSA G40.21 350W'
    Fy = 350*MPa
    Fu = 450*MPa

class HSS(Part):
    'HSS Column'
    grade = 'CSA G40.21 350W'
    Fy = 350*MPa
    Fu = 450*MPa
    size = 'HS127x127x13'
    D,T,A = SST.section(size,'D,T,A')
    D = D*mm
    T = T*mm
    A = A*mm*mm    

class CoverPlate(Plates):
    'Cover Plate'
    T = 10*mm
    W = 60*mm
    Lw = 90*mm      # length of weld from net section to end of HSS
    D = 6*mm        # size of weld from on HSS.
   
class TonguePlate(Plates):
    'Tongue Plate'
    T = 20*mm
    W = 280*mm
    L = 260*mm
    e = 40*mm
    D = 8*mm       # weld size
    c = 45*mm      # dist end of weld to 1st bolt line  
    Lw = L - (c + (Bolts.nlines-1)*Bolts.s + e)
    Dh = SST.section(HSS.size,'D')*mm
    ns = 10

class GussetPlate(TonguePlate['T,W'],Plates): 
    'Gusset Plate'
    W2 = 110*mm
    e = 40*mm    # end distance
    D = 8*mm     # plate to column weld size
    theta = 45. 

In [5]:
vars(Plates)

mappingproxy({__module__: __main__,
              __doc__: Plates,
              grade: CSA G40.21 350W,
              Fy: 350 <Unit('megapascal')>,
              Fu: 450 <Unit('megapascal')>})

In [6]:
vars(TonguePlate)

mappingproxy({__module__: __main__,
              __doc__: Tongue Plate,
              T: 20 <Unit('millimeter')>,
              W: 280 <Unit('millimeter')>,
              L: 260 <Unit('millimeter')>,
              e: 40 <Unit('millimeter')>,
              D: 8 <Unit('millimeter')>,
              c: 45 <Unit('millimeter')>,
              Lw: 100 <Unit('millimeter')>,
              Dh: 127.0 <Unit('millimeter')>,
              ns: 10})

In [7]:
TonguePlate.mro()

[__main__.TonguePlate, __main__.Plates, __main__.Part, object]

In [8]:
for o in TonguePlate.mro():
    print(o)
    print(o.__dict__)
    print()

<class '__main__.TonguePlate'>
{'__module__': '__main__', '__doc__': 'Tongue Plate', 'T': <Quantity(20, 'millimeter')>, 'W': <Quantity(280, 'millimeter')>, 'L': <Quantity(260, 'millimeter')>, 'e': <Quantity(40, 'millimeter')>, 'D': <Quantity(8, 'millimeter')>, 'c': <Quantity(45, 'millimeter')>, 'Lw': <Quantity(100, 'millimeter')>, 'Dh': <Quantity(127.0, 'millimeter')>, 'ns': 10}

<class '__main__.Plates'>
{'__module__': '__main__', '__doc__': 'Plates', 'grade': 'CSA G40.21 350W', 'Fy': <Quantity(350, 'megapascal')>, 'Fu': <Quantity(450, 'megapascal')>}

<class '__main__.Part'>
{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Part' objects>, '__weakref__': <attribute '__weakref__' of 'Part' objects>, '__doc__': None}

<class 'object'>
{'__repr__': <slot wrapper '__repr__' of 'object' objects>, '__hash__': <slot wrapper '__hash__' of 'object' objects>, '__str__': <slot wrapper '__str__' of 'object' objects>, '__getattribute__': <slot wrapper '__getattribute__' of 'object'

In [9]:
d = TonguePlate.__ns__()
d

{T: 20 <Unit('millimeter')>,
 W: 280 <Unit('millimeter')>,
 L: 260 <Unit('millimeter')>,
 e: 40 <Unit('millimeter')>,
 D: 8 <Unit('millimeter')>,
 c: 45 <Unit('millimeter')>,
 Lw: 100 <Unit('millimeter')>,
 Dh: 127.0 <Unit('millimeter')>,
 ns: 10,
 grade: CSA G40.21 350W,
 Fy: 350 <Unit('megapascal')>,
 Fu: 450 <Unit('megapascal')>}

In [10]:
d.clear?

Docstring: D.clear() -> None.  Remove all items from D.
Type:      builtin_function_or_method


In [11]:
Part.__dict__

mappingproxy({__module__: __main__,
              __dict__: <attribute '__dict__' of 'Part' objects>,
              __weakref__: <attribute '__weakref__' of 'Part' objects>,
              __doc__: None})

In [12]:
def SV(s,*objs,**kwds):
    ns = {}
    for o in reversed(objs):
        ns.update(o.__ns__())
    ns.update(kwds)
    return ns

In [15]:
SV('',CoverPlate,TonguePlate,LL=300)

{T: 10 <Unit('millimeter')>,
 W: 60 <Unit('millimeter')>,
 L: 260 <Unit('millimeter')>,
 e: 40 <Unit('millimeter')>,
 D: 6 <Unit('millimeter')>,
 c: 45 <Unit('millimeter')>,
 Lw: 90 <Unit('millimeter')>,
 Dh: 127.0 <Unit('millimeter')>,
 ns: 10,
 grade: CSA G40.21 350W,
 Fy: 350 <Unit('megapascal')>,
 Fu: 450 <Unit('megapascal')>,
 LL: 300}

In [16]:
def _setup(self):
    """Add all attributes/values to the set of global variables.
    Save enough state so that they can be restored when the context
    manager exits."""
    gns = get_ipython().user_ns  # get the ns for the user
    d = {}
    gns = get_ipython().user_ns
    for ob in self.objattrs:
        if ispartial(ob):
            for target,value in ob.ns().items():
                if target in d:
                    raise DesignerError('''Name '{}' has been used more than once.'''.format(target))
                d[target] = value
            continue
        else:
            obj,names = ob
        objns = obj.ns()
        for expr in se_split(names):
            if '=' in expr:
                target,rhs = [x.strip() for x in expr.split('=',1)]
                if rhs in objns:
                    value = objns[rhs]
                else:
                    value = eval(rhs,gns,objns)
            else:
                target = expr
                value = objns[expr]
            if target in d:
                raise DesignerError('''Name '{}' has been used more than once.'''.format(target))
            d[target] = value
    self.new_values = d
    self.local_vars = [y for y in [x.strip() for x in self.local.split(',')] if y] + [self.result_var]
    self.global_vars = [y for y in [x.strip() for x in self.globals.split(',')] if y]

    self.changed_values = {}
    self.added_vars = []
